In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
from evals import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

## Load

In [3]:
df = load_data('../data/ml-100k/u.data', threshold=3)
uuid = df['userId'].unique()
uiid = df['movieId'].unique()


In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df['userId'].values)

## Model

In [5]:
class AFM_layer(keras.Model):
    def __init__(self, x_dims, latent_dim, att_dim, l2_emb=1e-4):
        super(AFM_layer, self).__init__()
        self.x_dims = x_dims
        self.latent_dim = latent_dim
        self.att_dim = att_dim
        
        
        self.embedding = Embedding(sum(x_dims)+1, latent_dim, input_length=1, embeddings_regularizer=l2(l2_emb))
        
        self.linear = Dense(1)
        self.att_dense1 = Dense(att_dim)
        self.att_dense2 = Dense(1)
        self.final_dense = Dense(1)

    def call(self, inputs):
        cat_ = [tf.squeeze(tf.one_hot(feat, self.x_dims[i]), 1) for i, feat in enumerate(inputs)]
        X_cat = tf.concat(cat_, 1)
        X = tf.concat(inputs, 1)

        linear_terms = self.linear(X_cat)
        non_zero_emb = self.embedding(X + tf.constant((0, *np.cumsum(self.x_dims)))[:-1])

        n = len(self.x_dims)
        r = []; c = []
        for i in range(n-1):
            for j in range(i+1, n):
                r.append(i), c.append(j)
        p = tf.gather(non_zero_emb, r, axis=1)
        q = tf.gather(non_zero_emb, c, axis=1)
        pairwise = p*q
        
        att_score = tf.nn.relu(self.att_dense1(pairwise))
        att_score = tf.nn.softmax(self.att_dense2(att_score), axis=1)

        att_output = tf.reduce_sum(att_score * pairwise, axis=1)

        att_output = self.final_dense(att_output)
        
        y_hat = att_output + linear_terms

        return y_hat

In [6]:
class AFM(tf.keras.Model):
    def __init__(self, x_dim, latnt_dim, att_dim):
        super(AFM, self).__init__()
        self.afm = AFM_layer(x_dim, latnt_dim, att_dim)

    def call(self, inputs):
        outputs = self.afm(inputs)
#         outputs = tf.nn.sigmoid(outputs)
        return outputs

## Train

In [7]:
afm = AFM((len(uuid), len(uiid)), 8, 8)

In [8]:
afm.compile(loss=losses.BinaryCrossentropy(from_logits=True), optimizer=optimizers.Adam())
afm.fit([train['userId'].values.astype(np.int32), train['movieId'].values.astype(np.int32)], 
       train['rating'].values,
      epochs=10,
      shuffle=True,
      validation_split=0.1)

Epoch 1/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.6617 - val_loss: 0.6406
Epoch 2/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.6206 - val_loss: 0.6137
Epoch 3/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5962 - val_loss: 0.5980
Epoch 4/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5811 - val_loss: 0.5885
Epoch 5/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5711 - val_loss: 0.5824
Epoch 6/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5641 - val_loss: 0.5782
Epoch 7/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5590 - val_loss: 0.5755
Epoch 8/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5552 - val_loss: 0.5735
Epoch 9/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5522 - val_loss: 0.5721
Epoch 10/10
2391/2391 [==============================] - 10s 4ms/step - l

## Eval

In [9]:
pred = afm.predict([test['userId'].values.astype(np.int32), test['movieId'].values.astype(np.int32)])


In [10]:
np.sum(np.where(pred>0., 1, 0).flatten() == test['rating'].values) / len(pred)

0.7111333333333333

In [11]:
from sklearn.metrics import precision_score, recall_score,  roc_auc_score, precision_recall_fscore_support

print(roc_auc_score(test['rating'].values, pred))
print(precision_score(test['rating'].values, np.where(pred>0., 1, 0)))
print(recall_score(test['rating'].values, np.where(pred>0., 1, 0)))

0.7778323519046371
0.7173557325188175
0.79057465736956
